In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow.keras import Model, Sequential
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError

from tensorflow.keras.layers import Dense, Conv1D, LSTM, RNN

import warnings
warnings.filterwarnings('ignore')

In [26]:
dhaka_model = keras.models.load_model("lstm_dhaka_model.h5")

In [27]:
test_df = pd.read_csv('data/test.csv', index_col='datetime',parse_dates=True)


In [28]:
class DataWindow:
    def __init__(self, input_width, label_width, shift, test_df, label_columns=None):
        self.test_df = test_df
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}
        self.column_indices = {name: i for i, name in enumerate(test_df.columns)}
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift
        self.total_window_size = input_width + shift
        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]
        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def split_to_inputs_labels(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:, :, self.column_indices[name]] for name in self.label_columns],
                axis=-1
            )
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels

    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=32
        )

        ds = ds.map(self.split_to_inputs_labels)
        return ds

    @property
    def test(self):
        return self.make_dataset(self.test_df)


In [31]:
def predict(days: int):
    custom_mo_wide_window = DataWindow(input_width=days, label_width=days, shift=days, test_df=test_df,
                                       label_columns=['precip','rain_sum','river_discharge'])

    predicted_results = dhaka_model.predict(custom_mo_wide_window.test)
    predicted_array= predicted_results[0]

    predicted_numpy_array = np.array(predicted_array)

    df_scaled = pd.DataFrame(predicted_numpy_array)

    df = df_scaled.rename(columns={0: "river_discharge", 1: "rain_sum",2:"precip"})
    
    RD_max_train = 11.94 
    RD_min_train = 0.2
    RD_max_test =  28.11
    RD_min_test =  0.25

    R_max_train = 150.0
    R_min_train = 0.0
    R_max_test = 98.0
    R_min_test = 0.0

    P_max_train = 209.5
    P_min_train = 0.0
    P_max_test = 156.0
    P_min_test = 0.0


    df['river_discharge'] = df['river_discharge'].apply(lambda x: x*(RD_max_train - RD_min_train) + RD_min_train)
    df['rain_sum'] = df['rain_sum'].apply(lambda x: x*(R_max_train - R_min_train) + R_min_train)
    df['precip'] = df['precip'].apply(lambda x: x*(P_max_train - P_min_train) + P_min_train)
    df['floods'] = df['precip'] >2;
                      
    return df



In [32]:
predict(14)

12/12 [==============================] - 0s 3ms/step


,river_discharge,rain_sum,precip,floods
0,0.925927,5.606518,19.859038,True
1,0.984816,6.056246,19.981985,True
2,0.661790,2.689645,17.834363,True
3,0.660822,9.116089,19.872852,True
4,0.302718,1.190086,8.668363,True
5,1.048979,7.479926,17.503029,True
6,0.027901,2.008583,5.235043,True
7,0.996825,9.761278,14.072512,True
8,0.344315,5.016205,7.032777,True
9,0.709430,8.705640,9.433371,True
